In [1]:
!cp drive/My\ Drive/recsys.zip .

In [2]:
!unzip recsys.zip
!7z e members.csv.7z
!rm members.csv.7z
!7z e song_extra_info.csv.7z
!rm song_extra_info.csv.7z
!7z e songs.csv.7z
!rm songs.csv.7z
!7z e train.csv.7z
!rm train.csv.7z

Archive:  recsys.zip
  inflating: members.csv.7z          
  inflating: sample_submission.csv.7z  
  inflating: song_extra_info.csv.7z  
  inflating: songs.csv.7z            
  inflating: test.csv.7z             
  inflating: train.csv.7z            

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 1349856 bytes (1319 KiB)

Extracting archive: members.csv.7z
--
Path = members.csv.7z
Type = 7z
Physical Size = 1349856
Headers Size = 130
Method = LZMA2:3m
Solid = -
Blocks = 1

  0%    Everything is Ok

Size:       2503827
Compressed: 1349856

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for 

In [115]:
import pandas as pd
import numpy as np
from collections import defaultdict
from tqdm.autonotebook import tqdm
from multiprocessing import cpu_count
from gensim.models import Word2Vec
from functools import partial
from sklearn.metrics import roc_auc_score

In [5]:
train = pd.read_csv('train.csv')
songs = pd.read_csv('songs.csv')
songs_extra = pd.read_csv('song_extra_info.csv')

In [6]:
songs = pd.merge(songs, songs_extra, on='song_id', how='left')
songs.fillna('[unknown]', inplace=True)

Создадим для каждой песни новое имя - исполнитель: название песни

In [66]:
song_id2name = {}
for _, song in tqdm(songs.iterrows(), total=len(songs)):
    new_name = str(song['artist_name']) + ': ' + str(song['name'])
    song_id2name[song['song_id']] = new_name

Каждого юзера можно представить как среднее значение всех эмбеддингов песен, которые слушал этот юзер.

In [67]:
user2songs = defaultdict(list)
for _, sample in tqdm(train.iterrows(), total=len(train)):
    if sample['target'] == 0:
        continue

    if sample['song_id'] in song_id2name:
        user2songs[sample['msno']].append(song_id2name[sample['song_id']])

In [69]:
workers = cpu_count() - 1
word_model = Word2Vec(
    size=16,
    min_count=10,
    window=2,
    negative=10,
    sg=1,
    workers=workers
)
sentences = list(user2songs.values())

In [70]:
word_model.build_vocab(sentences)

In [71]:
word_model.train(sentences, 
                 total_examples=word_model.corpus_count, 
                 epochs=20)

(66880972, 74292200)

## Similars

In [72]:
word_model.init_sims(replace=True)

In [75]:
def get_similars(name, model=word_model, k=10):
    return model.wv.similar_by_word(name, topn=k)

In [77]:
get_similars('Eminem: Lose Yourself')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('Eminem: Not Afraid', 0.9707426428794861),
 ('Eminem: Berzerk', 0.955622673034668),
 ('Bad Meets Evil: Lighters', 0.9531140327453613),
 ('Eminem: Stan', 0.9467198848724365),
 ('B.o.B: Airplanes (feat. Hayley Williams)', 0.9456696510314941),
 ('Eminem: Headlights', 0.9445252418518066),
 ('Eminem: The Way I Am', 0.9372316002845764),
 ('Asher Monroe: Back for More (feat. Hot for Words)', 0.9330554008483887),
 ('Eminem: Rap God', 0.9326449632644653),
 ('Wiz Khalifa: No Sleep', 0.9319234490394592)]

Eminem предсказывается отлично, что неудивительно, так как он очень популярен. Посмотрим что-нибудь еще.

In [94]:
songs.iloc[random.randint(0, len(songs))]

song_id        +9NH3mIQyNcwMkTfibzNVUwLEDDSTjr20RbC/V+e6YE=
song_length                                          296054
genre_ids                                               465
artist_name                                 Michael Jackson
composer                                          [unknown]
lyricist                                          [unknown]
language                                                 52
name                                            Billie Jean
isrc                                           USSM10309701
Name: 1480705, dtype: object

In [95]:
get_similars('Michael Jackson: Billie Jean')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('Michael Jackson: Dangerous', 0.9670294523239136),
 ('Michael Jackson: Beat It', 0.9669744968414307),
 ('Michael Jackson: Smooth Criminal', 0.9620620608329773),
 ('Michael Jackson: Bad', 0.9526280760765076),
 ('Michael Jackson: Black Or White', 0.9470141530036926),
 ('Michael Jackson: The Way You Make Me Feel', 0.9467694163322449),
 ("Michael Jackson: I Just Can't Stop Loving You", 0.9342890977859497),
 ('Michael Jackson: Heal The World', 0.9323441982269287),
 ('Michael Jackson: Thriller', 0.9322249889373779),
 ('Michael Jackson: P.Y.T. (Pretty Young Thing)', 0.9253968000411987)]

Тут все еще более очевидно и хорошо :) Идем дальше

In [107]:
songs.iloc[random.randint(0, len(songs))]

song_id        TSkCFepLD3asIxKtwWqpm8rigPY6ZZ57WJwkW4DCSPc=
song_length                                          157338
genre_ids                                              1138
artist_name                                   Sylvia Telles
composer          Antônio Carlos Jobim| Aloysio De Oliveira
lyricist                                          [unknown]
language                                                 -1
name                                       De Você Eu Gosto
isrc                                           FRX761511993
Name: 1079974, dtype: object

In [108]:
get_similars('Sylvia Telles: De Você Eu Gosto')

KeyError: ignored

## Scores

In [118]:
user_embeddings = defaultdict(partial(np.zeros, 16))
for user in tqdm(user2songs):
    for song in user2songs[user]:
        try:
            user_embeddings[user] = user_embeddings[user] + word_model.wv.word_vec(song)
        except:
            continue

In [119]:
def get_user_embedding(user):
    return user_embeddings[user]

def get_song_embedding(song_id, model=word_model):
    try:
        return model.wv.word_vec(song_id2name[song_id]) 
    except:
        return np.zeros(16)

In [120]:
!pip install -q swifter
import swifter

In [121]:
song_embeddings = np.vstack(
                      train['song_id'].swifter.apply(get_song_embedding)
)
user_embeddings = np.vstack(
                      train['msno'].swifter.apply(get_user_embedding)
)

In [122]:
scores = (song_embeddings * user_embeddings).mean(axis=1)

In [123]:
roc_auc_score(train.target, scores)

0.6371341445650321

Catboost из прошлого задания выдал 0.755948. Разница довольно значительная. Но если нужен очень быстрый прототип рекомендательной системы или только симилары, то word2vec вполне неплохой вариант.